<a href="https://colab.research.google.com/github/mazarimono/webdb_notebook/blob/master/chapter-3_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3章
### COVID-19/家計調査/財政データの整形


In [1]:
import pandas as pd 

## 家計調査
### まずはデータを観測してどのようにデータを前処理するか方針を決める

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/mazarimono/webdb_notebook/master/data/h-mon-a.csv', encoding='shift-jis')
df.shape

(188, 251)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Columns: 251 entries, 二人以上の世帯_支出金額[円] to 245
dtypes: object(251)
memory usage: 368.8+ KB


In [4]:
df.head()

,二人以上の世帯_支出金額[円],Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245
0,NaN,NaN,NaN,NaN,NaN,NaN,2000年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020年,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,(平成12年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成13年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成14年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成30年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成31年),NaN,NaN,NaN,(令和元年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,(令和2年),NaN,NaN,NaN,NaN
2,表側連番,階層,大分類,中分類,小分類,品目分類,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,...,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月
3,1,-,-,-,-,世帯数分布(抽出率調整),10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
4,2,-,-,-,-,集計世帯数,7887,7942,7934,7922,7928,7917,7907,7908,7917,7937,7914,7933,7888,7887,7907,7927,7948,7924,7907,7909,7910,7908,7914,7924,7857,7877,7915,7894,7932,7926,7914,7893,7914,7933,...,7714,7689,7719,7735,7706,7710,7738,7729,7737,7678,7671,7596,7654,7656,7662,7648,7649,7648,7671,7617,7620,7600,7639,7555,7572,7581,7541,7548,7553,7501,7479,7517,7470,7466,7481,7443,7475,7489,7534,7430


In [5]:
df.tail()

,二人以上の世帯_支出金額[円],Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245
183,181,-,-,-,-,会費・つきあい費,1561,1441,1495,1888,1683,1585,1566,1390,1297,1390,1339,1603,1366,1374,1432,1735,1562,1519,1401,1269,1334,1241,1259,1467,1254,1324,1511,1778,1532,1453,1393,1161,1233,1252,...,693,806,1007,740,807,751,716,706,713,784,837,737,714,786,948,791,859,788,735,785,762,820,968,832,813,831,1063,878,929,864,707,802,710,776,860,770,705,690,751,696
184,182,-,-,-,-,その他の教養娯楽,7063,6370,8068,8109,8822,6858,7033,7995,6681,6831,7450,13030,7045,6272,7659,8213,9111,6615,6897,7755,6878,6794,7695,11965,7404,6274,8189,8577,9195,7119,7373,8510,7172,7335,...,7001,8999,9292,9880,8448,8278,9543,8197,7978,8806,12271,8103,7239,9220,8933,9453,7860,8634,9431,8375,8474,8814,11937,8646,7965,8659,9426,10263,8505,8641,9456,9369,8284,9152,12570,8239,7674,8995,8603,9612
185,183,-,-,-,-,消費支出(再掲),309621,290663,335341,335276,308566,297648,326480,309993,296457,309193,297144,391556,308912,293698,339910,318509,300346,286054,314251,302474,283973,305653,294929,359940,305221,273487,330380,320228,291968,289427,315699,298788,295423,304989,...,260644,297942,295929,283056,268802,279197,280320,268802,282872,277361,322157,289703,265614,301230,294439,281307,267641,283387,292481,271273,290396,281041,329271,296345,271232,309274,301136,300901,276882,288026,296327,300609,279671,278765,321380,287173,271735,292214,267922,252017
186,184,-,-,-,-,基礎的支出,184372,170045,186202,173441,177153,169779,180853,179332,166901,174690,170795,228730,188487,168993,184419,172758,174311,163180,175597,178844,164006,171481,170254,218506,184845,164207,183033,172079,172739,167985,178143,178849,168697,171674,...,153831,170012,163345,161531,158857,163808,166541,152786,158008,159545,195591,174274,157110,169880,160925,158522,156720,163681,169445,154120,161763,158922,194965,174007,155455,171966,157717,170407,161712,164379,170031,166350,159195,160265,190600,167873,157888,170401,159426,156494
187,185,-,-,-,-,選択的支出,125248,120618,149139,161835,131413,127868,145627,130661,129556,134503,126349,162826,120426,124705,155491,145751,126034,122873,138654,123630,119967,134172,124675,141434,120376,109280,147347,148148,119230,121442,137556,119939,126726,133315,...,106813,127931,132584,121525,109945,115389,113779,116016,124864,117815,126565,115429,108504,131351,133514,122785,110921,119706,123036,117153,128633,122118,134306,122337,115778,137308,143419,130494,115170,123648,126296,134260,120475,118500,130780,119301,113847,121813,108497,95523


### データを加工

In [6]:
# まずは列ごと取得してみる
year_data = df.loc[0].dropna()
year_data_list = [int(year.replace('年', '')) for year in list(year_data)]
end_month = df.iloc[2, -1]
end_month = int(end_month.replace('月', ''))
print(year_data_list)
print(end_month)

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
5


In [7]:
# 次に年月のデータを作成し、リストに格納する
month_count = (len(year_data_list) - 1) * 12 + end_month
date_list = pd.date_range('2000', periods=month_count, freq='M')

In [8]:
date_list[-5:]

DatetimeIndex(['2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30',
               '2020-05-31'],
              dtype='datetime64[ns]', freq='M')

In [9]:
kakei_data = df.iloc[3:, 5:]
kakei_data.head()

,Unnamed: 5,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245
3,世帯数分布(抽出率調整),10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
4,集計世帯数,7887,7942,7934,7922,7928,7917,7907,7908,7917,7937,7914,7933,7888,7887,7907,7927,7948,7924,7907,7909,7910,7908,7914,7924,7857,7877,7915,7894,7932,7926,7914,7893,7914,7933,7905,7932,7850,7879,7917,...,7714,7689,7719,7735,7706,7710,7738,7729,7737,7678,7671,7596,7654,7656,7662,7648,7649,7648,7671,7617,7620,7600,7639,7555,7572,7581,7541,7548,7553,7501,7479,7517,7470,7466,7481,7443,7475,7489,7534,7430
5,世帯人員(人),3.32,3.32,3.32,3.32,3.31,3.31,3.31,3.31,3.31,3.3,3.3,3.29,3.29,3.29,3.29,3.29,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.26,3.25,3.25,3.25,3.24,3.24,3.24,3.24,3.24,3.24,3.24,3.23,3.24,3.23,3.23,...,2.98,2.98,2.97,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.97,2.97,2.97,2.97,2.97,2.98,2.98,2.97,2.98,2.98,2.98,2.98,2.98,2.97,2.97,2.97,2.97,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96
6,18歳未満人員(人),0.74,0.75,0.75,0.75,0.75,0.74,0.74,0.74,0.74,0.73,0.72,0.73,0.72,0.72,0.72,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.7,0.69,0.7,0.69,0.69,0.7,0.7,0.7,0.7,0.71,0.71,0.7,0.69,0.69,0.69,0.68,...,0.54,0.54,0.54,0.55,0.55,0.55,0.54,0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.54,0.55,0.55,0.56,0.56,0.56,0.57,0.58,0.59,0.58,0.59,0.59,0.59,0.58,0.57,0.57,0.57,0.58,0.58,0.58,0.58,0.58,0.58,0.57,0.58,0.58
7,65歳以上人員(人),0.52,0.53,0.53,0.52,0.52,0.53,0.54,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.57,0.58,0.57,0.56,0.57,0.57,0.57,0.56,0.55,0.57,0.55,0.57,0.58,0.58,0.58,0.58,0.58,0.59,0.58,0.58,0.58,0.58,0.58,0.59,0.59,...,0.85,0.86,0.87,0.86,0.86,0.86,0.86,0.85,0.85,0.85,0.85,0.84,0.84,0.84,0.84,0.84,0.84,0.83,0.82,0.83,0.83,0.82,0.82,0.81,0.82,0.83,0.83,0.83,0.83,0.84,0.83,0.82,0.82,0.82,0.82,0.82,0.83,0.86,0.86,0.85


In [10]:
kakei_data = df.iloc[3:, 5:]
kakei_data = kakei_data.set_index('Unnamed: 5')
kakei_data.columns = date_list
kakei_data.index.name = ''
kakei_data = kakei_data.T
kakei_data['date'] = kakei_data.index 
kakei_data = kakei_data.reset_index(drop=True)
kakei_data

,世帯数分布(抽出率調整),集計世帯数,世帯人員(人),18歳未満人員(人),65歳以上人員(人),うち無職者人員(人),有業人員(人),世帯主の年齢(歳),持家率(％),家賃・地代を支払っている世帯の割合(％),消費支出,食料,穀類,米,パン,麺類,他の穀類,魚介類,生鮮魚介,塩干魚介,魚肉練製品,他の魚介加工品,肉類,生鮮肉,加工肉,乳卵類,牛乳,乳製品,卵,野菜・海藻,生鮮野菜,乾物・海藻,大豆加工品,他の野菜・海藻加工品,果物,生鮮果物,果物加工品,油脂・調味料,油脂,調味料,...,半耐久財(被服及び履物),サービス(被服及び履物),保健医療(財・サービス),耐久財(保健医療),半耐久財(保健医療),非耐久財(保健医療),サービス(保健医療),交通・通信(財・サービス),耐久財(交通・通信),半耐久財(交通・通信),非耐久財(交通・通信),サービス(交通・通信),教育(財・サービス),非耐久財(教育),サービス(教育),教養娯楽(財・サービス),耐久財(教養娯楽),半耐久財(教養娯楽),非耐久財(教養娯楽),サービス(教養娯楽),諸雑費(財・サービス),耐久財(諸雑費),半耐久財(諸雑費),非耐久財(諸雑費),サービス(諸雑費),調整集計世帯数,教養娯楽関係費,耐久財(教養娯楽関係費),他の教養娯楽関係費,読書,聴視・観覧,旅行,スポーツ,月謝,会費・つきあい費,その他の教養娯楽,消費支出(再掲),基礎的支出,選択的支出,date
0,10000,7887,3.32,0.74,0.52,0.41,1.51,52.4,76,22.1,309621,73580,6100,2338,2127,1325,310,8525,5479,1370,819,858,6229,5144,1085,3390,1590,1114,686,8029,4702,741,1250,1336,2866,2660,205,2897,264,2633,...,18712,891,10941,1123,422,3410,5986,31386,4435,1025,3876,22049,12532,281,12251,31681,4163,4624,6818,16075,19157,797,2064,4047,12249,736445,34008,3912,30096,4304,2049,8876,3001,3241,1561,7063,309621,184372,125248,2000-01-31
1,10000,7942,3.32,0.75,0.53,0.41,1.51,52.6,76.3,22,290663,73309,6915,2919,2329,1359,308,8627,5475,1500,802,850,6449,5242,1208,3650,1678,1200,772,8570,5102,744,1298,1427,3168,3001,167,3111,272,2838,...,13571,830,11789,853,443,3597,6896,31041,5455,887,3856,20843,14481,220,14260,29737,3026,3937,6892,15882,18800,731,2473,4291,11306,735391,32103,2920,29183,4586,2199,9058,2200,3329,1441,6370,290663,170045,120618,2000-02-29
2,10000,7934,3.32,0.75,0.53,0.41,1.51,52.7,76.2,22.6,335341,79726,7496,3226,2492,1482,296,9268,5880,1613,794,981,6483,5170,1314,3940,1835,1300,804,9268,5630,841,1311,1487,3270,3111,159,3262,308,2954,...,17174,1192,11767,1043,368,3891,6464,39097,9004,1542,4206,24345,17709,1100,16609,36010,4235,5050,8355,18370,22685,1006,2816,4896,13968,735404,39415,4248,35166,4858,1977,12249,3022,3497,1495,8068,335341,186202,149139,2000-03-31
3,10000,7922,3.32,0.75,0.52,0.41,1.52,52.6,75.8,22.2,335276,77344,7470,3346,2441,1458,225,8736,5438,1571,761,966,6485,5082,1404,3803,1769,1264,770,9268,5762,828,1262,1415,2999,2848,152,3149,315,2834,...,16141,1887,11251,1000,286,3340,6624,41356,10981,1563,4087,24725,24049,926,23124,34146,2890,4777,8519,17960,20400,580,2216,4528,13076,734606,37458,3109,34349,4552,2470,10432,3129,3768,1888,8109,335276,173441,161835,2000-04-30
4,10000,7928,3.31,0.75,0.52,0.41,1.53,52.7,76.1,22.6,308566,81415,7447,3264,2458,1523,203,8857,5554,1586,756,961,6655,5168,1487,3899,1877,1258,763,9533,5983,767,1314,1469,3148,2985,162,3307,321,2985,...,16753,1960,11335,965,513,3370,6487,36089,6613,1158,4467,23852,11512,427,11084,33824,2517,3666,9032,18608,19449,452,2321,4698,11979,733789,38525,3354,35171,4516,2202,11746,2946,3256,1683,8822,308566,177153,131413,2000-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,10000,7443,2.96,0.58,0.82,0.65,1.34,59.3,85.6,13.1,287173,76011,5666,1402,2515,1335,414,5723,3350,990,635,748,7162,5913,1250,3661,1158,1766,737,7832,5102,659,1055,1016,2800,2478,322,3345,339,3006,...,11578,606,13880,979,235,4986,7681,38918,5004,2002,4981,26931,8719,66,8653,28174,2870,3561,5140,16602,25060,518,2542,4929,17071,655791,30567,2995,27572,3187,2590,7299,2839,2648,770,8239,287173,167873,119301,2020-01-31
241,10000,7475,2.96,0.58,0.83,0.66,1.33,59.3,85.3,13.4,271735,75469,6266,1779,2625,1442,421,5548,3141,991,627,789,7261,5952,1309,3846,1176,1891,779,8081,5192,719,1108,1062,3009,2721,288,3603,366,3237,...,7800,524,14236,834,414,5468,7520,42390,10988,2553,4991,23858,8392,127,8265,25597,1736,3066,5088,15707,22372,331,1679,5189,15173,655490,27381,1696,25684,3184,3149,5922,2436,2614,705,7674,271735,157888,113847,2020-02-29
242,10000,7489,2.96,0.57,0.86,0.68,1.32,59.5,85.7,13.3,292214,79509,725

In [12]:
!mkdir data
kakei_data.to_csv('data/kakei_data.csv')

In [13]:
kakei = pd.read_csv('data/kakei_data.csv', index_col=0)
kakei.tail()

,世帯数分布(抽出率調整),集計世帯数,世帯人員(人),18歳未満人員(人),65歳以上人員(人),うち無職者人員(人),有業人員(人),世帯主の年齢(歳),持家率(％),家賃・地代を支払っている世帯の割合(％),消費支出,食料,穀類,米,パン,麺類,他の穀類,魚介類,生鮮魚介,塩干魚介,魚肉練製品,他の魚介加工品,肉類,生鮮肉,加工肉,乳卵類,牛乳,乳製品,卵,野菜・海藻,生鮮野菜,乾物・海藻,大豆加工品,他の野菜・海藻加工品,果物,生鮮果物,果物加工品,油脂・調味料,油脂,調味料,...,半耐久財(被服及び履物),サービス(被服及び履物),保健医療(財・サービス),耐久財(保健医療),半耐久財(保健医療),非耐久財(保健医療),サービス(保健医療),交通・通信(財・サービス),耐久財(交通・通信),半耐久財(交通・通信),非耐久財(交通・通信),サービス(交通・通信),教育(財・サービス),非耐久財(教育),サービス(教育),教養娯楽(財・サービス),耐久財(教養娯楽),半耐久財(教養娯楽),非耐久財(教養娯楽),サービス(教養娯楽),諸雑費(財・サービス),耐久財(諸雑費),半耐久財(諸雑費),非耐久財(諸雑費),サービス(諸雑費),調整集計世帯数,教養娯楽関係費,耐久財(教養娯楽関係費),他の教養娯楽関係費,読書,聴視・観覧,旅行,スポーツ,月謝,会費・つきあい費,その他の教養娯楽,消費支出(再掲),基礎的支出,選択的支出,date
240,10000,7443,2.96,0.58,0.82,0.65,1.34,59.3,85.6,13.1,287173,76011,5666,1402,2515,1335,414,5723,3350,990,635,748,7162,5913,1250,3661,1158,1766,737,7832,5102,659,1055,1016,2800,2478,322,3345,339,3006,...,11578,606,13880,979,235,4986,7681,38918,5004,2002,4981,26931,8719,66,8653,28174,2870,3561,5140,16602,25060,518,2542,4929,17071,655791,30567,2995,27572,3187,2590,7299,2839,2648,770,8239,287173,167873,119301,2020-01-31
241,10000,7475,2.96,0.58,0.83,0.66,1.33,59.3,85.3,13.4,271735,75469,6266,1779,2625,1442,421,5548,3141,991,627,789,7261,5952,1309,3846,1176,1891,779,8081,5192,719,1108,1062,3009,2721,288,3603,366,3237,...,7800,524,14236,834,414,5468,7520,42390,10988,2553,4991,23858,8392,127,8265,25597,1736,3066,5088,15707,22372,331,1679,5189,15173,655490,27381,1696,25684,3184,3149,5922,2436,2614,705,7674,271735,157888,113847,2020-02-29
242,10000,7489,2.96,0.57,0.86,0.68,1.32,59.5,85.7,13.3,292214,79509,7254,2132,2798,1825,498,6336,3599,1117,655,965,8155,6612,1544,4286,1336,2063,888,9111,5935,803,1196,1177,3193,2895,298,4050,451,3599,...,9313,643,14143,894,542,5135,7571,47231,12780,2697,4697,27056,10530,805,9725,24996,2672,3823,6035,12466,25201,588,1514,5548,17551,654527,25464,2668,22796,3405,2225,2693,2742,2046,690,8995,292214,170401,121813,2020-03-31
243,10000,7534,2.96,0.58,0.86,0.68,1.33,59.5,85.8,13.2,267922,73919,7253,2110,2680,1921,542,6136,3477,1133,632,894,8610,7002,1609,4463,1367,2165,931,10215,6930,853,1245,1187,3031,2724,307,4201,464,3737,...,4525,674,12894,526,375,5851,6142,39111,8951,2462,3559,24139,19278,457,18821,21123,2350,2989,5689,10095,20478,230,869,5076,14302,653331,21433,2344,19090,3244,2992,590,1494,1416,751,8603,267922,159426,108497,2020-04-30
244,10000,7430,2.96,0.58,0.85,0.68,1.33,59.6,85.7,12.9,252017,78272,7059,1921,2742,1909,487,6423,3734,1164,629,896,9115,7405,1710,4609,1472,2184,954,10400,7182,790,1233,1195,3040,2753,287,4106,472,3634,...,7012,767,12322,647,333,5914,5427,34248,6473,1707,3376,22692,9288,299,8989,20325,1981,3617,6298,8428,20542,253,827,5193,14268,652743,20956,1968,18988,3250,2035,798,1703,893,696,9612,252017,156494,95523,2020-05-31


## 新型コロナ感染者数データ

In [14]:
pd.set_option('max_columns', 60)
csv_url = 'https://dl.dropboxusercontent.com/s/6mztoeb6xf78g5w/COVID-19.csv'
df = pd.read_csv(csv_url, low_memory=False)
print(df.shape)
print(df.info())

(59983, 52)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59983 entries, 0 to 59982
Data columns (total 52 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   通し                59983 non-null  int64  
 1   厚労省NO             3694 non-null   object 
 2   無症状病原体保有者         910 non-null    object 
 3   国内                302 non-null    object 
 4   チャーター便            11 non-null     object 
 5   年代                59983 non-null  object 
 6   性別                59983 non-null  object 
 7   確定日               59983 non-null  object 
 8   発症日               26862 non-null  object 
 9   受診都道府県            59983 non-null  object 
 10  居住都道府県            59983 non-null  object 
 11  居住                327 non-null    object 
 12  居住市区町村            36046 non-null  object 
 13  キー                59983 non-null  object 
 14  発表                59981 non-null  object 
 15  都道府県内症例番号         55839 non-null  object 
 16  市町村内症例番号          10626 non-

In [15]:
df.head()

,通し,厚労省NO,無症状病原体保有者,国内,チャーター便,年代,性別,確定日,発症日,受診都道府県,居住都道府県,居住,居住市区町村,キー,発表,都道府県内症例番号,市町村内症例番号,ステータス,備考,ソース,ソース2,ソース3,人数,累計,前日比,発症数,死者合計,退院数累計,退院数,PCR検査実施人数,PCR検査前日比,職業_正誤確認用,勤務先_正誤確認用,Hospital Pref,Residential Pref,Release,Gender,X,Y,確定日YYYYMMDD,受診都道府県コード,居住都道府県コード,更新日時,Field2,Field4,Field5,Field6,Field7,Field8,Field9,Field10,Unnamed: 51
0,1,1,NaN,A-1,NaN,30,男性,1/15/2020,1/3/2020,神奈川県,神奈川県,NaN,NaN,神奈川県,神奈川県,1,NaN,退院,NaN,https://www.mhlw.go.jp/stf/newpage_08906.html,https://www.pref.kanagawa.jp/docs/ga4/bukanshi...,NaN,1,1.0,1.0,0.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,Kanagawa,Kanagawa,Kanagawa Prefecture,Male,139.642347,35.447504,2020/1/15,14.0,14.0,8/23/2020 02:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,NaN,A-2,NaN,40,男性,1/24/2020,1/14/2020,東京都,中華人民共和国,NaN,NaN,中華人民共和国,東京都,1,NaN,退院または死亡,NaN,https://www.mhlw.go.jp/stf/newpage_09079.html,https://www.metro.tokyo.lg.jp/tosei/hodohappyo...,NaN,1,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tokyo,China(Mainland),Tokyo Metropolitan Government,Male,116.409685,39.903832,2020/1/24,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,NaN,A-3,NaN,30,女性,1/25/2020,1/21/2020,東京都,中華人民共和国,NaN,NaN,中華人民共和国,東京都,2,NaN,退院または死亡,NaN,https://www.mhlw.go.jp/stf/newpage_09099.html,https://www.metro.tokyo.lg.jp/tosei/hodohappyo...,NaN,1,3.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tokyo,China(Mainland),Tokyo Metropolitan Government,Female,116.409685,39.903832,2020/1/25,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,NaN,A-4,NaN,40,男性,1/26/2020,1/23/2020,愛知県,中華人民共和国,NaN,NaN,中華人民共和国,愛知県,1,NaN,NaN,国籍:中国,https://www.mhlw.go.jp/stf/newpage_09100.html,https://www.pref.aichi.jp/uploaded/attachment/...,NaN,1,4.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aichi,China(Mainland),Aichi Prefecture,Male,116.409685,39.903832,2020/1/26,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5,NaN,A-5,NaN,40,男性,1/28/2020,1/22/2020,愛知県,中華人民共和国,NaN,NaN,中華人民共和国,愛知県,2,NaN,NaN,国籍:中国,https://www.mhlw.go.jp/stf/newpage_09154.html,https://www.pref.aichi.jp/uploaded/attachment/...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aichi,China(Mainland),Aichi Prefecture,Male,116.409685,39.903832,2020/1/28,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df.tail()

,通し,厚労省NO,無症状病原体保有者,国内,チャーター便,年代,性別,確定日,発症日,受診都道府県,居住都道府県,居住,居住市区町村,キー,発表,都道府県内症例番号,市町村内症例番号,ステータス,備考,ソース,ソース2,ソース3,人数,累計,前日比,発症数,死者合計,退院数累計,退院数,PCR検査実施人数,PCR検査前日比,職業_正誤確認用,勤務先_正誤確認用,Hospital Pref,Residential Pref,Release,Gender,X,Y,確定日YYYYMMDD,受診都道府県コード,居住都道府県コード,更新日時,Field2,Field4,Field5,Field6,Field7,Field8,Field9,Field10,Unnamed: 51
59978,59455,NaN,NaN,NaN,NaN,不明,不明,8/22/2020,NaN,東京都,東京都,NaN,NaN,東京都,東京都,NaN,NaN,NaN,NaN,https://www3.nhk.or.jp/news/html/20200822/k100...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tokyo,Tokyo,Tokyo Metropolitan Government,Unknown,139.691647,35.689185,2020/8/22,13.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59979,59456,NaN,NaN,NaN,NaN,不明,不明,8/22/2020,NaN,東京都,東京都,NaN,NaN,東京都,東京都,NaN,NaN,NaN,NaN,https://www3.nhk.or.jp/news/html/20200822/k100...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tokyo,Tokyo,Tokyo Metropolitan Government,Unknown,139.691647,35.689185,2020/8/22,13.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59980,59457,NaN,NaN,NaN,NaN,不明,不明,8/22/2020,NaN,東京都,東京都,NaN,NaN,東京都,東京都,NaN,NaN,NaN,NaN,https://www3.nhk.or.jp/news/html/20200822/k100...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tokyo,Tokyo,Tokyo Metropolitan Government,Unknown,139.691647,35.689185,2020/8/22,13.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59981,59978,NaN,NaN,NaN,NaN,80,女性,8/22/2020,8/21/2020,北海道,北海道,後志総合振興局,NaN,北海道後志総合振興局,北海道,1691,NaN,NaN,NaN,NaN,http://www.pref.hokkaido.lg.jp/hf/kth/kak/kisy...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hokkaido,Hokkaido,Hokkaido Prefecture,Female,140.755551,42.901660,2020/8/22,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59982,59979,NaN,NaN,NaN,NaN,不明,不明,8/22/2020,8/18/2020,北海道,北海道,オホーツク総合振興局,NaN,北海道オホーツク総合振興局,北海道,1692,NaN,NaN,NaN,NaN,http://www.pref.hokkaido.lg.jp/hf/kth/kak/kisy...,NaN,1,59983.0,260.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hokkaido,Hokkaido,Hokkaido Prefecture,Unknown,144.260865,44.026645,2020/8/22,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
selected_cols = ['年代', '性別', '確定日', '発症日', '居住都道府県', 'X', 'Y']
df1 = df[selected_cols]
df1.head()

,年代,性別,確定日,発症日,居住都道府県,X,Y
0,30,男性,1/15/2020,1/3/2020,神奈川県,139.642347,35.447504
1,40,男性,1/24/2020,1/14/2020,中華人民共和国,116.409685,39.903832
2,30,女性,1/25/2020,1/21/2020,中華人民共和国,116.409685,39.903832
3,40,男性,1/26/2020,1/23/2020,中華人民共和国,116.409685,39.903832
4,40,男性,1/28/2020,1/22/2020,中華人民共和国,116.409685,39.903832


In [18]:
df1.tail()

,年代,性別,確定日,発症日,居住都道府県,X,Y
59978,不明,不明,8/22/2020,NaN,東京都,139.691647,35.689185
59979,不明,不明,8/22/2020,NaN,東京都,139.691647,35.689185
59980,不明,不明,8/22/2020,NaN,東京都,139.691647,35.689185
59981,80,女性,8/22/2020,8/21/2020,北海道,140.755551,42.901660
59982,不明,不明,8/22/2020,8/18/2020,北海道,144.260865,44.026645


In [19]:
df1 = df1.dropna(how='all')
df1.tail()

,年代,性別,確定日,発症日,居住都道府県,X,Y
59978,不明,不明,8/22/2020,NaN,東京都,139.691647,35.689185
59979,不明,不明,8/22/2020,NaN,東京都,139.691647,35.689185
59980,不明,不明,8/22/2020,NaN,東京都,139.691647,35.689185
59981,80,女性,8/22/2020,8/21/2020,北海道,140.755551,42.901660
59982,不明,不明,8/22/2020,8/18/2020,北海道,144.260865,44.026645


In [20]:
df1['count'] = 1
df1

,年代,性別,確定日,発症日,居住都道府県,X,Y,count
0,30,男性,1/15/2020,1/3/2020,神奈川県,139.642347,35.447504,1
1,40,男性,1/24/2020,1/14/2020,中華人民共和国,116.409685,39.903832,1
2,30,女性,1/25/2020,1/21/2020,中華人民共和国,116.409685,39.903832,1
3,40,男性,1/26/2020,1/23/2020,中華人民共和国,116.409685,39.903832,1
4,40,男性,1/28/2020,1/22/2020,中華人民共和国,116.409685,39.903832,1
...,...,...,...,...,...,...,...,...
59978,不明,不明,8/22/2020,NaN,東京都,139.691647,35.689185,1
59979,不明,不明,8/22/2020,NaN,東京都,139.691647,35.689185,1
59980,不明,不明,8/22/2020,NaN,東京都,139.691647,35.689185,1
59981,80,女性,8/22/2020,8/21/2020,北海道,140.755551,42.901660,1


In [21]:
for col in df1.columns:
    print(f'{col}: {df1[col].unique()}')

年代: ['30 ' '40 ' '60 ' '50 ' '20 ' '80 ' '70 ' '10 ' '0-10' '不明' '90 ' '90'
 '60' '70' '30' '40' '20' '50' '80' '10']
性別: ['男性' '女性' '不明' '男性\xa0' '女性\xa0']
確定日: ['1/15/2020' '1/24/2020' '1/25/2020' '1/26/2020' '1/28/2020' '1/29/2020'
 '1/30/2020' '1/31/2020' '2/1/2020' '2/4/2020' '2/5/2020' '2/7/2020'
 '2/10/2020' '2/11/2020' '2/13/2020' '2/14/2020' '2/15/2020' '2/16/2020'
 '2/17/2020' '2/18/2020' '2/19/2020' '2/20/2020' '2/21/2020' '2/22/2020'
 '2/23/2020' '2/24/2020' '2/25/2020' '2/26/2020' '2/27/2020' '2/28/2020'
 '2/29/2020' '3/1/2020' '3/2/2020' '3/3/2020' '3/4/2020' '3/5/2020'
 '3/6/2020' '3/7/2020' '3/8/2020' '3/9/2020' '3/10/2020' '3/11/2020'
 '3/12/2020' '3/13/2020' '3/14/2020' '3/15/2020' '3/16/2020' '3/17/2020'
 '3/18/2020' '3/19/2020' '3/20/2020' '3/21/2020' '3/22/2020' '3/23/2020'
 '3/24/2020' '3/25/2020' '3/26/2020' '3/27/2020' '3/28/2020' '3/29/2020'
 '3/30/2020' '3/31/2020' '4/1/2020' '4/2/2020' '4/3/2020' '4/4/2020'
 '4/5/2020' '4/6/2020' '4/7/2020' '4/8/2020' '4/9/20

In [22]:
df1['年代'].astype('str').apply(lambda x: x.replace('0-10', '1'))

0        30 
1        40 
2        30 
3        40 
4        40 
        ... 
59978     不明
59979     不明
59980     不明
59981    80 
59982     不明
Name: 年代, Length: 59983, dtype: object

In [23]:
df1['年代'] = df1['年代'].apply(lambda x: x.replace('0-10', '1'))
df1['年代'] = df1['年代'].apply(lambda x: x.replace('不明', '50'))
df1['年代'] = df1['年代'].apply(lambda x: x.replace('nan', '50'))

df1['年代'] = df1['年代'].apply(lambda x: int(x))
df1['年代'].unique()

array([30, 40, 60, 50, 20, 80, 70, 10,  1, 90])

In [24]:
df1.to_csv('data/covid_19_update.csv')

## 日本の財政データ

In [25]:
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)

excel_url = 'https://www.mof.go.jp/budget/reference/statistics/24.xls'
df = pd.read_excel(excel_url)
print(df.shape)
print(df.info())

WARNING *** file size (44668) not 512 + multiple of sector size (512)
(57, 26)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   17 non-null     object 
 1   Unnamed: 1   37 non-null     object 
 2   Unnamed: 2   37 non-null     object 
 3   Unnamed: 3   0 non-null      float64
 4   Unnamed: 4   53 non-null     object 
 5   Unnamed: 5   53 non-null     object 
 6   Unnamed: 6   53 non-null     object 
 7   Unnamed: 7   53 non-null     object 
 8   Unnamed: 8   53 non-null     object 
 9   Unnamed: 9   53 non-null     object 
 10  Unnamed: 10  53 non-null     object 
 11  Unnamed: 11  53 non-null     object 
 12  Unnamed: 12  53 non-null     object 
 13  Unnamed: 13  53 non-null     object 
 14  Unnamed: 14  53 non-null     object 
 15  Unnamed: 15  53 non-null     object 
 16  Unnamed: 16  53 non-null     object 
 17  Unnamed: 17  53

In [26]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,第24表 平 成 ９ 年 度 以 降 一 般 会 計 歳 出 決 算 目 的 別 分 類,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,（単位：千円）
3,年 度\n目 的 別,NaN,NaN,NaN,平 成 ９ 年 度,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df.tail()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
52,11.東日本大震災復旧・復興予備費,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,ー,ー,ー,ー,ー,－
53,12.熊本地震復旧等予備費,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,ー,ー,ー,ー,ー,－
54,12．予 備 費,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,ー,ー,ー,ー,ー,－
55,13．そ の 他,NaN,NaN,NaN,107446364,560720538,1946185898,117848770,114811714,116061688,131099215,137464675,134327404,132535572,126513456,596003290,3972807075,1998855325,1151337298,3127364837,4049324814,1944845682,1306542097,1017311678,950583079,940610781
56,合 計,NaN,NaN,NaN,78470310184,84391798185,89037431392,89321049910,84811128504,83674289924,82415970662,84896776304,85519592207,81445480694,81842570225,84697395043,100973424388,95312341707,100715409134,97087176621,100188878834,98813467430,98230324099,97541764842,98115604721,98974696544


In [28]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,第24表 平 成 ９ 年 度 以 降 一 般 会 計 歳 出 決 算 目 的 別 分 類,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,（単位：千円）
3,年 度\n目 的 別,NaN,NaN,NaN,平 成 ９ 年 度,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,１．国 家 機 関 費,NaN,NaN,NaN,4089878933,4280839099,4273674613,4802768587,4845892598,4308629662,4382520701,4464313211,4460200856,4361194867,4530742429,4470410404,5074090225,4945147236,4928503182,4369555684,4867349264,4860631635,5061873526,4979956434,4825663205,4807859792
6,NaN,⑴,皇室費,NaN,6686157,6643501,6832632,9366209,7408714,7089358,6721568,6965778,6849417,6591482,6569224,6613183,6137774,5491398,6026580,5965628,5774431,6151561,5800414,5351842,6776077,9678519
7,NaN,⑵,国会費,NaN,123019403,131142130,138585747,146463865,154158813,133486528,127362960,127144108,123122693,122817737,122136741,122660956,126258243,151426912,135321370,128495174,125668672,132043907,134301666,132560410,133287519,136393763
8,NaN,⑶,選挙費,NaN,3534438,57491037,3258220,73263020,63135863,5853557,71775157,61379786,69799237,1328890,59141967,702891,68788324,52100534,319976,59452810,50432786,56412418,1643113,53738213,59832965,143469
9,NaN,⑷,司法、警察及び消防費,NaN,1319365876,1399227861,1408610493,1453162305,1410079489,1386978225,1390638515,1384424712,1411240550,1431981688,1468295392,1496556058,1533909368,1472709759,1597209034,1480602910,1524285156,1502880959,1514804508,1573470081,1560110218,1629272958


In [29]:
df1 = df.iloc[5:56, :]
df1 = df1.drop(['Unnamed: 1', 'Unnamed: 3'], axis=1)
df1 = df1.reset_index(drop=True)
df1.columns = [f'col_{i}' for i in range(len(df1.columns))]

In [30]:
df1.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23
0,１．国 家 機 関 費,NaN,4089878933,4280839099,4273674613,4802768587,4845892598,4308629662,4382520701,4464313211,4460200856,4361194867,4530742429,4470410404,5074090225,4945147236,4928503182,4369555684,4867349264,4860631635,5061873526,4979956434,4825663205,4807859792
1,NaN,皇室費,6686157,6643501,6832632,9366209,7408714,7089358,6721568,6965778,6849417,6591482,6569224,6613183,6137774,5491398,6026580,5965628,5774431,6151561,5800414,5351842,6776077,9678519
2,NaN,国会費,123019403,131142130,138585747,146463865,154158813,133486528,127362960,127144108,123122693,122817737,122136741,122660956,126258243,151426912,135321370,128495174,125668672,132043907,134301666,132560410,133287519,136393763
3,NaN,選挙費,3534438,57491037,3258220,73263020,63135863,5853557,71775157,61379786,69799237,1328890,59141967,702891,68788324,52100534,319976,59452810,50432786,56412418,1643113,53738213,59832965,143469
4,NaN,司法、警察及び消防費,1319365876,1399227861,1408610493,1453162305,1410079489,1386978225,1390638515,1384424712,1411240550,1431981688,1468295392,1496556058,1533909368,1472709759,1597209034,1480602910,1524285156,1502880959,1514804508,1573470081,1560110218,1629272958


In [31]:
print(df1.iloc[46, 4:].unique())
print(df1.iloc[47, 4:].unique())
print(df1.iloc[48, 4:].unique())
print(df1.iloc[49, 4:].unique())

['-' 'ー' '－']
['-' 'ー' '－']
['-' 'ー' '－']
['-' 'ー' '－']


In [32]:
replace_strs = df1.iloc[46, 4:].unique()
df1 = df1.replace(replace_strs, 0)

In [33]:
df1.tail()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23
46,10.経済危機対応・地域活性化予備費,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
47,11.東日本大震災復旧・復興予備費,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48,12.熊本地震復旧等予備費,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49,12．予 備 費,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50,13．そ の 他,NaN,107446364,560720538,1946185898,117848770,114811714,116061688,131099215,137464675,134327404,132535572,126513456,596003290,3972807075,1998855325,1151337298,3127364837,4049324814,1944845682,1306542097,1017311678,950583079,940610781


In [34]:
df1['main_title'] = df1['col_0'].fillna(method='ffill')
df1.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,main_title
0,１．国 家 機 関 費,NaN,4089878933,4280839099,4273674613,4802768587,4845892598,4308629662,4382520701,4464313211,4460200856,4361194867,4530742429,4470410404,5074090225,4945147236,4928503182,4369555684,4867349264,4860631635,5061873526,4979956434,4825663205,4807859792,１．国 家 機 関 費
1,NaN,皇室費,6686157,6643501,6832632,9366209,7408714,7089358,6721568,6965778,6849417,6591482,6569224,6613183,6137774,5491398,6026580,5965628,5774431,6151561,5800414,5351842,6776077,9678519,１．国 家 機 関 費
2,NaN,国会費,123019403,131142130,138585747,146463865,154158813,133486528,127362960,127144108,123122693,122817737,122136741,122660956,126258243,151426912,135321370,128495174,125668672,132043907,134301666,132560410,133287519,136393763,１．国 家 機 関 費
3,NaN,選挙費,3534438,57491037,3258220,73263020,63135863,5853557,71775157,61379786,69799237,1328890,59141967,702891,68788324,52100534,319976,59452810,50432786,56412418,1643113,53738213,59832965,143469,１．国 家 機 関 費
4,NaN,司法、警察及び消防費,1319365876,1399227861,1408610493,1453162305,1410079489,1386978225,1390638515,1384424712,1411240550,1431981688,1468295392,1496556058,1533909368,1472709759,1597209034,1480602910,1524285156,1502880959,1514804508,1573470081,1560110218,1629272958,１．国 家 機 関 費


In [35]:
col0_has_value = df1['col_0'].dropna()
df_big = df1.iloc[col0_has_value.keys(), :].dropna(1)
col_data = [year for year in range(1997, 2019)]
col_data.insert(0, 'title')
col_data.insert(len(col_data), 'main_title')
df_big.columns = col_data
df_big = df_big.reset_index(drop=True)

In [36]:
col0_has_value

0            １．国 家 機 関 費
9            ２．地 方 財 政 費
13           ３．防 衛 関 係 費
14           ４．国土保全及び開発費
20           ５．産 業 経 済 費
26           ６．教 育 文 化 費
31             ７．社会保障関係費
41             ８．恩　　給　　費
45             ９．国　　債　　費
46    10.経済危機対応・地域活性化予備費
47     11.東日本大震災復旧・復興予備費
48         12.熊本地震復旧等予備費
49            12．予　　備　　費
50            13．そ　　の　　他
Name: col_0, dtype: object

In [37]:
df_big.head()

,title,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,main_title
0,１．国 家 機 関 費,4089878933,4280839099,4273674613,4802768587,4845892598,4308629662,4382520701,4464313211,4460200856,4361194867,4530742429,4470410404,5074090225,4945147236,4928503182,4369555684,4867349264,4860631635,5061873526,4979956434,4825663205,4807859792,１．国 家 機 関 費
1,２．地 方 財 政 費,15503917972,14326238212,13105011723,15849461435,16725670735,16500834094,17427158365,17703484772,17504074308,16761090449,15010305478,15702928407,16595904253,18809852561,19469488558,16900990748,17575309002,17185775269,16888314210,15421261402,15644801858,16099323993,２．地 方 財 政 費
2,３．防 衛 関 係 費,4975677697,4982414986,4923268867,4931354728,4993022486,4943816744,4951477751,4921492949,4900978100,4838999823,4778565357,4823873909,4831771933,4678220145,4824861715,4770980200,4802875442,5073428675,5141147921,5162012628,5286811260,5487851505,３．防 衛 関 係 費
3,４．国土保全及び開発費,9210198515,11052792039,11083969090,10236076779,9272010124,7963231929,8248139969,8055495328,7942207471,6937891934,6576189506,6314345042,7562815928,5670372003,6422748987,5761729805,7887098236,7218035719,6387967080,6746035521,6894955678,6971474840,４．国土保全及び開発費
4,５．産 業 経 済 費,3240075324,4910396599,4011527035,4111681798,4026447612,5421252999,3213295564,3248391168,2998228230,2734879729,3159778905,3949330597,7643915744,4265026290,6661729113,4827678295,3563750596,3509288639,3368098750,3602752975,3157471367,3309277871,５．産 業 経 済 費


In [38]:
col1_has_value = df1['col_1'].dropna()
df_detail = df1.iloc[col1_has_value.keys(), :].dropna(1)
df_detail.columns = col_data
df_add = df_big.iloc[[2, 8, 9, 10, 11, 12, 13], :]
df_detail = pd.concat([df_detail, df_add])
df_detail = df_detail.reset_index(drop=True)

In [39]:
df_detail.head()

,title,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,main_title
0,皇室費,6686157,6643501,6832632,9366209,7408714,7089358,6721568,6965778,6849417,6591482,6569224,6613183,6137774,5491398,6026580,5965628,5774431,6151561,5800414,5351842,6776077,9678519,１．国 家 機 関 費
1,国会費,123019403,131142130,138585747,146463865,154158813,133486528,127362960,127144108,123122693,122817737,122136741,122660956,126258243,151426912,135321370,128495174,125668672,132043907,134301666,132560410,133287519,136393763,１．国 家 機 関 費
2,選挙費,3534438,57491037,3258220,73263020,63135863,5853557,71775157,61379786,69799237,1328890,59141967,702891,68788324,52100534,319976,59452810,50432786,56412418,1643113,53738213,59832965,143469,１．国 家 機 関 費
3,司法、警察及び消防費,1319365876,1399227861,1408610493,1453162305,1410079489,1386978225,1390638515,1384424712,1411240550,1431981688,1468295392,1496556058,1533909368,1472709759,1597209034,1480602910,1524285156,1502880959,1514804508,1573470081,1560110218,1629272958,１．国 家 機 関 費
4,外交費,831845677,861739833,888920739,865866900,864050579,810000539,844190419,905133307,836553817,847587839,880421187,914480244,955076436,879664492,821449422,783527090,850286829,870024473,906512055,924167345,888687574,878467212,１．国 家 機 関 費


In [40]:
df_detail.to_csv('data/zaisei_detail.csv')